In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyspark
!pip install faiss-cpu --no-cache
!pip install autofaiss

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 47 kB/s 
     |████████████████████████████████| 199 kB 44.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845514 sha256=4576e053f6d3c9daffaf275aa23b0e0aec8d4cce40e5d1734acfed30f67b76e9
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.6 MB 6.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 4.3 MB/s 
     |████████████████████████████████| 87 kB 5.6 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115940 sha256=6358680b2a851a4b2a658f88686d6bef8b7834

In [ ]:
import numpy as np
import pandas as pd
import sys
import time
import os
import shutil
import faiss
import glob
import numpy as np
from pyspark.sql import SparkSession

import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from collections import Counter
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import time
from operator import add
from sklearn.neighbors import KNeighborsClassifier

dataset


In [ ]:
from sklearn.datasets import fetch_covtype
forest = fetch_covtype()
Data= forest['data']
label = forest['target']
data_cov = np.concatenate([Data[:, :32], label.reshape(-1, 1)], axis = -1)

In [ ]:
data_cov = data_cov[:10000, :]
data_cov.shape

(10000, 33)

In [ ]:
all_cov, query_cov = train_test_split(data_cov, test_size=0.2, random_state=4)

MR baseline

In [ ]:
def Distance(test, train):
    '''
    Input:
        - test: Numpy Array with label as last value
        - train: Numpy Array with label as last value
    
    Returns distance between test and train arrays
    '''
    # dist = np.sqrt(np.sum((test[:-1]-train[:-1])**2))
    # int(train[-1]), np.sum((test[:-1] - train[:-1]) ** 2)
    return np.sum(np.subtract(test[:-1],train[:-1]) ** 2), int(train[-1])

def MR_baseline(all_data_np, query_data_np, K):
    print('current K = ', K)
    spark = SparkSession\
            .builder\
            .appName("PythonKNN")\
            .getOrCreate()
    
    # spark.conf.set("spark.sql.shuffle.partitions", "2")
    sc = spark.sparkContext

    sc.setLogLevel("ERROR")

    df = pd.DataFrame(all_data_np)
    all_data = spark.createDataFrame(df).rdd
    all_data = all_data.map(lambda x: np.array(x))

    df = pd.DataFrame(query_data_np)
    query_data = spark.createDataFrame(df).rdd
    query_data = query_data.map(lambda x: np.array(x))

    start_time = time.time()
    
    accuracy = 0
    count = 0
    for test_point in query_data.collect():

        distances = all_data.map(
            lambda train_point: Distance(test_point, train_point))

        # # print('distances = ', distances)
        # for d in distances.collect():
        #     print('d =', d)

        k_nearest_neighbours = sc.parallelize(distances.takeOrdered(K, key = lambda p: p[0])).map(lambda x: (x[1], 1))
        
        k_nearest_predictions = k_nearest_neighbours.reduceByKey(
            lambda x1, x2: x1 + x2)
        
        predict_label = k_nearest_predictions.takeOrdered(1,
            key = lambda x: -x[1])[0][0]

        if predict_label == test_point[-1]:
            accuracy += 1

        count += 1

    end_time = time.time()
    print('accuracy = ', accuracy / count)
    time_taken = end_time - start_time
    print('time_taken = ', time_taken)

In [ ]:
for k in [8, 16,32,64]:
  MR_baseline(all_cov, query_cov, k)

current K =  8
accuracy =  0.767
time_taken =  854.0006308555603
current K =  16
accuracy =  0.736
time_taken =  833.285843372345
current K =  32
accuracy =  0.708
time_taken =  844.1968381404877
current K =  64
accuracy =  0.6535
time_taken =  829.3452432155609
